### csv 파일 출처
* [나이스 교육정보 개방 포털](https://open.neis.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=OPEN13920190311110530306647&infSeq=1#none)

In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [2]:

# 위도 경도 구하기 ---- kako api
def get_coordinate(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + 'cba6eb29be7c48e97d9a5cd246429cdd'}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

### 행정구역_지도
![행정구역_지도](../img/%ED%96%89%EC%A0%95%EA%B5%AC%EC%97%AD_%EC%A7%80%EB%8F%84.png)
* [출처 | 위키백과](https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%ED%96%89%EC%A0%95_%EA%B5%AC%EC%97%AD)

In [3]:
# 광역시, 특별시, 특별자치시, 특별자치도 - 다른 행정구역에 소속되지 않음 ( 광역시와 행정구역을 분리하여 생각 )

# 1특별시, 6광역시, 1특별자치시, 8도, 1특별자치도
# => 17개의 데이터 파일 생성 

# html 이름은 csv 파일의 '교육청'빼고 설정

In [4]:
# 학교 정보 크롤링(네이버)
error = []

def crawling_high_school_data(high_school_name) -> list:

    url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query={high_school_name}'
 
    response = requests.get(url)

    html = response.text

    soup = BeautifulSoup( html, 'html.parser' )

    # 0 : 주소
    # 1 : 전화번호
    # 2 : 개교 연도
    # 3 : 학생수
    a_tag = soup.select('.home') # 홈페이지 주소
    data  = soup.select('dd') # 주소, 전화번호, 개교 연도, 학생수 등

    # 리스트에 담음
    list_data = []
    for i in data[:4]:
        list_data.append(i.text)

    # 홈페이지 주소 추가
    link = []
    for i in a_tag:
        link.append(i.attrs['href'])
        
    list_data.append(link[0])

    # 0 : 주소
    # 1 : 전화번호
    # 2 : 개교 연도
    # 3 : 학생수
    # 4 : 홈페이지
    return list_data
    

In [5]:
crawling_high_school_data('대정고등학교')

['제주특별자치도 서귀포시 대정읍 일주서로2498번길 21',
 ' 064-730-0100 ',
 '1952년 06월 02일',
 '293명 남 293명, 여 0명 ',
 'http://daejeong.jje.hs.kr']

In [6]:
def put_high_school_data(high_school_name, affiliations='') -> dict:
    high_school_data = crawling_high_school_data(high_school_name)
    
    # index
    # 0 : 주소
    # 1 : 전화번호
    # 2 : 개교 연도
    # 3 : 학생수
    # 4 : 홈페이지
    address = high_school_data[0]
    phone = high_school_data[1].strip(' ')
    birth = high_school_data[2]

    # index
    # 0 : 총 학생수
    # 2 : 남 학생수
    # 4 : 여 학생수
    students_data = high_school_data[3].split(' ')

    # 0 : 전체 학생수 / 2 : 남학생 수 / 4 : 여학생 수
    if len(students_data) <= 2: # 하지만 split한 list의 길이가 2 이하이면 총 학생수만
        total_students = students_data[0]
        boys            = '0명'
        girls           = '0명'
    else:
        total_students = students_data[0]
        boys            = students_data[2].strip(',')
        girls           = students_data[4]

    # 경도, 위도
    longitude, latitude = get_coordinate(high_school_data[0])
    
    link = high_school_data[4]

    result = {
        '주소' : address,
        '위도' : latitude,
        '경도' : longitude,
        '전화번호' : phone,
        '재학생 수' : total_students,
        '남학생 수' : boys,
        '여학생 수' : girls,
        '홈페이지' : link,
        '계열' : affiliations
    }

    return result

    

In [7]:
put_high_school_data('대정고등학교', '일반계')

{'주소': '제주특별자치도 서귀포시 대정읍 일주서로2498번길 21',
 '위도': 33.2319799989913,
 '경도': 126.259500374588,
 '전화번호': '064-730-0100',
 '재학생 수': '293명',
 '남학생 수': '293명',
 '여학생 수': '0명',
 '홈페이지': 'http://daejeong.jje.hs.kr',
 '계열': '일반계'}

In [8]:

# 딕셔너리에 집어넣고 csv 파일 만듦

def make_csv():
    path = '../code/교육청_csv/'

    file_list = os.listdir(path) # 폴더 안 파일 목록 가져옴

    offices_of_education_csv = file_list

    for i in offices_of_education_csv:
        df = pd.read_csv(path + i)

        high_school_names = list(set(df['학교명']))

        high_schools_data = {}

        for j in high_school_names:
            # 학교 계열
            _list = list(df[df['학교명'] == j]['계열명']) # 특정 학교 계열명 추출
            affiliations = ''
            try:
                for k in _list:
                    affiliations += k + ' '
                    if _list[-1] != k:
                        affiliations += ' | '
                        
                high_schools_data[j] = put_high_school_data(j, affiliations) 
            except:
                error.append(j)
                continue
                
        high_school_dataframe = pd.DataFrame(high_schools_data).T
        csv_name = i.strip('.csv') + '고등학교데이터.csv'
        csv_path = '../data_csv/' + csv_name
        high_school_dataframe.to_csv(csv_path)


In [9]:
len(error)

0

In [15]:
# csv 파일 재설정 
# - unnamed: 0열 지우기
# - 다른 지역들에 같은 고등학교 때문에 고등학교 이름 앞에 행정구역의 이름을 붙여준다.
#    ->  그다음 다시 데이터를 집어넣어 값을 수정

def reset_csv():
    data_csv_path = '../data_csv/'

    file_list = os.listdir(data_csv_path) # 폴더 안 파일 목록 가져옴

    data_csv_list = file_list

    for i in data_csv_list:
        df = pd.read_csv(data_csv_path + i)
        
        # Unnamed: 0열을 지우고 재설정
        save_index = list(df['Unnamed: 0'])
        df.drop(['Unnamed: 0'], axis=1, inplace=True)
        df.index = save_index

        # 이름 앞에 행정구역의 이름 붙여줌
        head_word = i.strip('교육청고등학교데이터.csv')
        
        high_school_addresses = list(df['주소'])

        high_school_names = df.index
        
        for j in range(len(high_school_addresses)):
            if head_word not in high_school_addresses[j]:
                # 크롤링한 데이터를 바꿈
                # 주소, 전화번호, 재학생, 남학생, 여학생, 위도, 경도 바꿈
                try:
                    new_name = head_word + high_school_names[j]

                    new_info = put_high_school_data(new_name)
                    df.at[high_school_names[j], '주소'] = new_info['주소']
                    df.at[high_school_names[j], '위도'] = new_info['위도']
                    df.at[high_school_names[j], '경도'] = new_info['경도']
                    df.at[high_school_names[j], '전화번호'] = new_info['전화번호']
                    df.at[high_school_names[j], '재학생 수'] = new_info['재학생 수']
                    df.at[high_school_names[j], '남학생 수'] = new_info['남학생 수']
                    df.at[high_school_names[j], '여학생 수'] = new_info['여학생 수']
                    df.at[high_school_names[j], '홈페이지'] = new_info['홈페이지']
                    df.rename(index={high_school_names[j] : new_name}, inplace=True)

                    print(head_word + high_school_names[j])
                except:
                    df.drop(high_school_names[j], inplace=True)
                    continue
        df.to_csv(data_csv_path + i) # 저장

In [ ]:
make_csv() # csv 만듦

In [16]:
reset_csv() # csv 재설정

강원도인제고등학교
강원도성수고등학교
강원도석정여자고등학교
강원도대화고등학교
강원도대성고등학교
강원도대진고등학교
강원도상동고등학교
강원도설악고등학교
경기도중산고등학교
경기도홍익디자인고등학교
경기도오산고등학교
경기도광남고등학교
경기도한광고등학교
경기도영신여자고등학교
경기도용문고등학교
경기도청담고등학교
경기도대신고등학교
경기도백암고등학교
경기도기흥고등학교
경기도성남고등학교
경기도광문고등학교
경기도양진중학교
경기도인창고등학교
경상남도용남고등학교
경상남도명덕고등학교
경상남도영산고등학교
경상남도고성고등학교
경상남도세종고등학교
경상남도웅천고등학교
경상남도한일여자고등학교
경상남도경상고등학교
경상남도대곡고등학교
경상남도동원고등학교
경상남도덕산고등학교
경상북도영덕고등학교
경상북도대영고등학교
경상북도율곡고등학교
경상북도영일고등학교
경상북도유성여자고등학교
경상북도영양여자고등학교
경상북도세화고등학교
경상북도세명고등학교
경상북도금천고등학교
경상북도경일고등학교
경상북도경안고등학교
경상북도영동고등학교
경상북도풍산고등학교
광주광역시성덕고등학교
광주광역시보문고등학교
광주광역시동명고등학교
광주광역시성덕중학교
광주광역시상일여자고등학교
광주광역시광덕고등학교
광주광역시광주고등학교
광주광역시금호고등학교
대구광역시대원고등학교
대구광역시수성고등학교
대구광역시강동고등학교
대구광역시오성고등학교
대구광역시계성고등학교
대구광역시운암고등학교
대구광역시영신고등학교
대구광역시대진고등학교
대구광역시경화여자고등학교
대구광역시경신고등학교
대구광역시구암고등학교
대전광역시서일고등학교
대전광역시중일고등학교
부산광역시대동고등학교
부산광역시금곡고등학교
부산광역시경원고등학교
부산광역시광명고등학교
부산광역시영산고등학교
부산광역시성지고등학교
부산광역시신도고등학교
부산광역시양정고등학교
부산광역시대광고등학교
부산광역시경일고등학교
부산광역시지산고등학교
부산광역시신정고등학교
부산광역시용인고등학교
부산광역시금성고등학교
부산광역시혜화여자고등학교
부산광역시부흥고등학교
서울특별시영신고등학교
서울특별시광남고등학교
서울특별시언남고등학교
세종특별자치시장기

In [15]:
# 로고 주소 공식
# 홈페이지 주소 + / + img src